# Tarea breve 2

## Diplomado en Ciencia de Datos para las Ciencias Sociales y las Políticas Públicas.

#### Integrantes
- Alvaro Zapata
- Diego Delgado


## Parte I
#### 1. ¿Qué es un proceso embarrassingly parallel y uno inherentemente serial? Dé un ejemplo de cada uno (distintos a los vistos en clase)

Un proceso embarrassingly parallel consiste en ejecutar tareas que pueden dividirse en partes independientes sin necesidad de comunicación entre ellas, como el procesamiento de imágenes en un gran conjunto de fotos. Por el contrario, un proceso inherentemente serial requiere una ejecución secuencial específica, como la ejecución de un algoritmo donde cada paso depende del resultado del paso anterior.




#### 2. ¿Cuáles son los dos principales cuellos de botella al paralelizar un proceso? Explíquelo a partir de las leyes de Amdahl y Gustafson 

Los principales cuellos de botella son la porción serial del programa, según la Ley de Amdahl, y la eficiencia de la paralelización que disminuye con el aumento de la porción paralelizable, según la Ley de Gustafson. La Ley de Amdahl dice que el tiempo de ejecución está limitado por la parte no paralelizable del programa, mientras que la Ley de Gustafson se enfoca en cómo la escalabilidad se ve afectada por la eficiencia de la paralelización.



#### 3. Describa los recursos (CPU y GPU) de su computadora y provea evidencia (ie. screenshot). 
No tengo GPU solo CPU.


 



#### 4. ¿En qué se diferencia un CPU de un GPU? Dé un ejemplo de un proceso que convendría paralelizar en cada uno de estos tipos de unidad de procesamiento. 

Un CPU está diseñado para realizar una amplia gama de tareas y es adecuado para operaciones seriales, como la ejecución de un navegador web. En cambio, un GPU está optimizado para tareas paralelas, como el renderizado de gráficos en videojuegos. Por ejemplo, sería optimo paralelizar el procesamiento gráfico (como videojuegos) en un GPU, mientras que la ejecución de algoritmos de búsqueda sería adecuada para un CPU.




#### 5. Se sabe estima que la nueva memoria RAM POWER 3000 permitirá acelerar las operaciones de su computadora en la mitad del tiempo. ¿Qué tanto más rápido será la velocidad de ejecución de un programa si el 77% del mismo se dedica a operaciones inherentemente paralelizables?

Si el 77% de un programa es paralelizable y la nueva memoria permite acelerar las operaciones a la mitad del tiempo, la aceleración total del programa se puede estimar usando la Ley de Amdahl.
S=1/((1-P)+P/N)
Donde P es la fracción paralelizable y N es el factor de aceleración de la memoria.
S=1/((1-0.77)+0.77/2)≈1.85



#### 6. Piense en una tarea serial que le han encargado paralelizar. Describa el diseño de la implementación en paralelo de dicha tarea siguiente el método de Foster y los cuatro elementos que lo componen.

Si por ejemplo queremos ordenar una lista larga, se puede usar método de Foster. Primero, se puede dividir la lista en sublistas más pequeñas. Luego, se le asigna estas sublistas a diferentes procesadores. Despues, calcular la ordenación de las sublistas en paralelo. El último paso sería combinar las sublistas ordenadas en una lista final ordenada.



#### 7. ¿Cuál es la diferencia entre point-to-point communication y collective communication en MPI? ¿Cuál es la diferencia entre las operaciones de broadcasting, gathering y scattering?

La comunicación point-to-point implica la transferencia de datos entre dos procesos individuales específicos, utilizando funciones como `MPI_Send` y `MPI_Recv`. En cambio, la comunicación colectiva involucra a todos los procesos de un grupo y utiliza operaciones como `MPI_Bcast`, `MPI_Gather` y `MPI_Scatter`, que afectan a todos los procesos simultáneamente. Broadcasting (`MPI_Bcast`) permite que un proceso envíe el mismo dato a todos los demás procesos, gathering (`MPI_Gather`) permite que cada proceso envíe sus diferentes datos al proceso raíz para consolidarlos, y scattering (`MPI_Scatter`), lo contrario al Gather, distribuye segmentos desde el proceso raíz a los demás procesos para procesamiento paralelo.

## Parte II
### 1. Código de paralelización de impresión de listas

#### a y b. Generamso el código y lo ejecutamos en diferentes procesadores 

In [26]:
#!pip install mpi4py
import os

os.chdir(r'C:\Users\DDelgado\Documents\GitHub\Big_Data_Analytics_QLAB\code')

In [37]:
%%writefile parte2_1.py
from mpi4py import MPI


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    # Procesador 0 genera la lista y la envía a los otros 3 procesadores
    data = list(range(10))  # Lista de ejemplo
    for i in range(1, 4):
        comm.send(data, dest=i, tag=11)
        print(f"Procesador 0 envió la lista al procesador {i}")
elif rank != 0:
    # Procesadores 1, 2 y 3 reciben la lista y la imprimen
    data = comm.recv(source=0, tag=11)
    print(f"Procesador {rank} recibió la lista: {data}")

Overwriting parte2_1.py


In [38]:
!mpiexec -n 4 python parte2_1.py

Procesador 1 recibió la lista: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Procesador 0 envió la lista al procesador 1
Procesador 0 envió la lista al procesador 2
Procesador 0 envió la lista al procesador 3
Procesador 2 recibió la lista: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Procesador 3 recibió la lista: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


##### c. De ejecutar el código 100 veces, ¿el orden de los resultados será siempre igual? ¿Por qué?
De ejecutar el código 100 veces, el orden de los resultados no será siempre igual. Esto se debe a que, aunque el procesador 0 envía los mensajes en un orden específico, la recepción e impresión de los mensajes por parte de los otros procesadores puede ocurrir en un orden no determinista. El entorno de MPI y la planificación del sistema operativo pueden hacer que los procesadores 1, 2 y 3 reciban y ejecuten las instrucciones de impresión en diferentes órdenes en distintas ejecuciones.

### 2. Broadcasting y scattering

Escribimos un código que haga lo siguiente:

**Parte a: Broadcasting:**

- El procesador 0 define el diccionario cursos_favoritos.
- Usamos comm.bcast para transmitir el diccionario a todos los procesadores.
- Registramos y comparamos el tiempo de demora en cada procesador.

**Parte b: Scattering:**

- El procesador 0 define una secuencia de valores valores que va de 0 a size - 1.
- Usamos comm.Scatter para distribuir estos valores entre los procesadores.
- Cada procesador recibe un valor y comprueba si coincide con su rango (rank).

In [43]:
%%writefile parte2_2.py

from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Parte a: Broadcasting
cursos_favoritos = {
    0: "Econometría avanzada",
    1: "Introduccion al Machine Learning",
    2: "Estadística",
    3: "Python Intermedio"
}

# Solo el procesador 0 define el diccionario
if rank == 0:
    data = cursos_favoritos
else:
    data = None

# Registrar el tiempo antes del broadcasting
start_time = time.time()

# Broadcasting del diccionario
data = comm.bcast(data, root=0)

# Registrar el tiempo después del broadcasting
end_time = time.time()
broadcast_time = end_time - start_time

# Imprimir el diccionario recibido y el tiempo de demora en cada procesador
print(f"Procesador {rank} recibió el diccionario: {data} en {broadcast_time:.6f} segundos")

# Parte b: Scattering
# Definir una secuencia de valores {0, 1, 2, ..., n}
if rank == 0:
    valores = np.arange(size, dtype='i')
else:
    valores = None

# Preparar un array para recibir el valor dispersado
valor_recibido = np.zeros(1, dtype='i')

# Dispersar los valores
comm.Scatter(valores, valor_recibido, root=0)

# Imprimir el valor recibido y comprobar si coincide con el rango del procesador
print(f"Procesador {rank} recibió el valor {valor_recibido[0]}. Coincide con el rango: {valor_recibido[0] == rank}")


Overwriting mpi_broadcast_scatter.py


Ejecutamos el broadcasting:

In [42]:
!mpiexec -n 4 python parte2_2.py


Procesador 0 recibió el diccionario: {0: 'Econometría avanzada', 1: 'Introduccion al Machine Learning', 2: 'Estadística', 3: 'Python Intermedio'} en 0.001029 segundos
Procesador 0 recibió el valor 0. Coincide con el rango: True
Procesador 1 recibió el diccionario: {0: 'Econometría avanzada', 1: 'Introduccion al Machine Learning', 2: 'Estadística', 3: 'Python Intermedio'} en 0.000000 segundos
Procesador 1 recibió el valor 1. Coincide con el rango: True
Procesador 3 recibió el diccionario: {0: 'Econometría avanzada', 1: 'Introduccion al Machine Learning', 2: 'Estadística', 3: 'Python Intermedio'} en 0.003033 segundos
Procesador 3 recibió el valor 3. Coincide con el rango: True
Procesador 2 recibió el diccionario: {0: 'Econometría avanzada', 1: 'Introduccion al Machine Learning', 2: 'Estadística', 3: 'Python Intermedio'} en 0.002033 segundos
Procesador 2 recibió el valor 2. Coincide con el rango: True


### 3. Análisis de estadísiticas de array

#### a. Definir numpy

In [53]:
import numpy as np
import pandas as pd
import time

# Cargar el archivo CSV
data = pd.read_csv(r'../data/tarea2.csv')
values = data.to_numpy()

# Registrar el tiempo antes de encontrar el máximo
start_time = time.time()

# Hallar el valor máximo
max_value = np.max(values)

# Registrar el tiempo después de encontrar el máximo
end_time = time.time()
single_processor_time = end_time - start_time

print(f"Valor máximo usando un solo procesador: {max_value}")
print(f"Tiempo de demora: {single_processor_time:.6f} segundos")


Valor máximo usando un solo procesador: 99.74206107
Tiempo de demora: 0.000552 segundos


#### b. Paralelizacion de cálculo de stats

El código a continuación realiza lo siguiente:

- (i) dividir el numpy en dos partes iguales
- (ii) que dos procesadores distintos encuentren el máximo de cada parte 
- (iii) que otro procesador junte los máximos hallados y encuentre el máximo global. Registrar el tiempo de demora.

In [61]:
%%writefile parte2_3a.py

from mpi4py import MPI
import numpy as np
import pandas as pd
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Solo se necesitan 3 procesadores para esta tarea
if size != 3:
    raise ValueError("Este programa requiere exactamente 3 procesadores.")

# Cargar el archivo CSV y dividir los datos en dos partes
if rank == 0:
    data = pd.read_csv(r'../data/tarea2.csv')
    values = data.to_numpy()
    part1 = values[:len(values) // 2]
    part2 = values[len(values) // 2:]
else:
    part1 = None
    part2 = None

# Broadcasting de las partes
part1 = comm.bcast(part1, root=0)
part2 = comm.bcast(part2, root=0)

# Hallar el máximo en cada parte
if rank == 1:
    local_max = np.max(part1)
    print(f"Procesador {rank} encontró el máximo local: {local_max}")
elif rank == 2:
    local_max = np.max(part2)
    print(f"Procesador {rank} encontró el máximo local: {local_max}")
else:
    local_max = None

# Asegurarse de que local_max no sea None
if local_max is None:
    raise ValueError(f"Procesador {rank} no calculó un máximo local válido.")

# El procesador 0 reúne los máximos hallados y encuentra el máximo global
global_max = comm.reduce(local_max, op=MPI.MAX, root=0)

if rank == 0:
    end_time = time.time()
    print(f"Valor máximo usando dos procesadores: {global_max}")
    print(f"Tiempo de demora: {end_time - start_time:.6f} segundos")

Overwriting parte2_3a.py


In [62]:
!mpiexec -n 3 python parte2_3a.py


^C
